# Extraction des textes dans des documents de formats divers
---

## Installation des dépendances
---

In [19]:
# @title
!pip install markitdown[pdf] --quiet
!pip install PyPDF2 --quiet
!pip install pymupdf4llm --quiet
!pip install easyocr --quiet
!pip install tiktoken --quiet
!pip install verovio --quiet
!pip install mistralai --quiet
!pip install -U openai-whisper --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [24]:
import warnings
warnings.filterwarnings("ignore")

## Extraire les textes issus des documents pdf
---

In [25]:
# Bouton pour uploader un fichier au début du notebook
from google.colab import files
uploaded = files.upload()

Saving Acceuil_affichage_Mairie de Triffouillis sur Loire.pdf to Acceuil_affichage_Mairie de Triffouillis sur Loire (3).pdf


In [26]:
# Chemin vers le fichier PDF
pdf_path = "/content/Acceuil_affichage_Mairie de Triffouillis sur Loire.pdf"

### Markitdown

In [ ]:
from markitdown import MarkItDown

md = MarkItDown()
result = md.convert(pdf_path)
print(result.text_content)

### PyPDF2

In [27]:
import PyPDF2

# Ouvrir le fichier PDF en mode lecture binaire
with open(pdf_path, "rb") as pdf_file:
    # Créer un lecteur PDF
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Extraire le texte de chaque page
    text = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()

    # Afficher le texte extrait
    print(text)

 
Mairie de Triffouillis sur Loire  
Avis d’Information et Horaires d’Ouverture  
 
I. Message d’Accueil  
Chers concitoyens,  
Nous avons le plaisir de vous accueillir sous l’égide de la bonne humeur et du sérieux 
nécessaire à l’exercice de nos fonctions. La Mairie de Triffouillis sur Loire vous invite à 
consulter les informations ci -après afin de mieux connaîtr e nos services et modalités de 
fonctionnement.  
Nous vous présentons, en tête d’affiche, notre maire officieux et attachant : Madame 
Pétillante Rigolade , qui, avec un sourire communicatif, veille à la bonne marche de notre 
commune. Son engagement, conjugué à l’expertise de nos équipes, vous assure une 
prise en charge rapide et efficace de vos demandes.  
 
II. Horaires d’Ouverture et Contacts  
Afin de vous servir au mieux, notre mairie est ouverte selon des horaires fixes, adaptés 
aux besoins de chacun. Vous trouverez ci -dessous un tableau récapitulatif des horaires 
de nos services. Pour toute information compléme

### Pymupdf4llm

In [28]:
import pymupdf4llm

md_text = pymupdf4llm.to_markdown(pdf_path)
print("=== Contenu Markdown extrait avec PyMuPDF4LLM ===")
print(md_text)

=== Contenu Markdown extrait avec PyMuPDF4LLM ===
# **Mairie de Triffouillis sur Loire ** **Avis d’Information et Horaires d’Ouverture** **I. Message d’Accueil ** Chers concitoyens, Nous avons le plaisir de vous accueillir sous l’égide de la bonne humeur et du sérieux nécessaire à l’exercice de nos fonctions. La Mairie de Triffouillis sur Loire vous invite à consulter les informations ci-après afin de mieux connaître nos services et modalités de fonctionnement. **Madame ** Nous vous présentons, en tête d’affiche, notre maire officieux et attachant : Pétillante Rigolade, qui, avec un sourire communicatif, veille à la bonne marche de notre commune. Son engagement, conjugué à l’expertise de nos équipes, vous assure une prise en charge rapide et efficace de vos demandes. **II. Horaires d’Ouverture et Contacts ** Afin de vous servir au mieux, notre mairie est ouverte selon des horaires fixes, adaptés aux besoins de chacun. Vous trouverez ci-dessous un tableau récapitulatif des horaires de n

## Océriser les textes contenus dans les images

---



In [29]:
# Bouton pour uploader un fichier au début du notebook
from google.colab import files
uploaded = files.upload()

Saving marche_noel_trifouillis_2023.png to marche_noel_trifouillis_2023 (1).png


In [30]:
image_path = "/content/marche_noel_trifouillis_2023.png"

### Easy OCR

In [31]:
import easyocr
import os

# Initialiser EasyOCR
reader = easyocr.Reader(['fr'])  # 'fr' pour le français, ajoutez d'autres langues si nécessaire

# Extraire le texte de chaque image
full_text = ""

# Appliquer l'OCR sur l'image
results = reader.readtext(image_path)

# Formater les résultats
for (bbox, text, prob) in results:
    full_text += text + "\n"
print(full_text)

(RIFOULLIS
Mairie de Triffouillis-sur-Loire
Organisation du Marché de Noël 2023
La commune de Triffouillis-sur-Loire est heureuse d'annoncer l'édition 2023 de son
célèbre Marché de Noël
Cette année encore; petits et grands pourront se plonger dans
magie des fêtes en découvrant de nombreux exposants et animations festives.
Date
Samedi 16 et dimanche 17 décembre 2023
Lieu
Place du Grand Chêne
Horaires
10h00
20h00
Les exposants présents
Les Délices de Tante Gertrude -
Spécialités alsaciennes
pains
épices
maison
La Fabrique du Père Noël _ Jouets en bois faits main
Vins
Saveurs d'Hiver - Dégustation de vins chauds et produits du terroir
Les Lutins du Crochet - Bonnets
echarpes et
tricoresavec anour
La Forge des Rennes
Sculptures métalliques et décorations artisanales
Choco-Miam
Fontaine
chocolat et friandises gourmandes
Le Fromage qui Chante
Sélection de fromages affinés avec démonstration
musicale
La Boutique des Licornes de Noël-
Cadeaux féériques
accessoires
enchantes
Monsieur Marcel
ses

### GOT OCR

In [ ]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True)
model = AutoModel.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True, low_cpu_mem_usage=True, device_map='cuda', use_safetensors=True, pad_token_id=tokenizer.eos_token_id)

# plain texts OCR
res = model.chat(tokenizer, image_path, ocr_type='ocr')

print(res)

### Mistral Document Understanding

In [32]:
import base64
import requests
import os
from mistralai import Mistral

api_key = "StthE16bt5IReMHPxgjbwtQLGggjOGtq"



def encode_image(image_path):
    """Encode the image to base64."""
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"Error: The file {image_path} was not found.")
        return None
    except Exception as e:  # Added general exception handling
        print(f"Error: {e}")
        return None

# Getting the base64 string
base64_image = encode_image(image_path)

client = Mistral(api_key=api_key)

ocr_response = client.ocr.process(
    model="mistral-ocr-latest",
    document={
        "type": "image_url",
        "image_url": f"data:image/jpeg;base64,{base64_image}"
    }
)
ocr_response

OCRResponse(pages=[OCRPageObject(index=0, markdown="La commune de Triffouillis-sur-Loire est heureuse d'annoncer l'édition 2023 de son célèbre Marché de Noël ! Cette année encore, petits et grands pourront se plonger dans la magie des fêtes en découvrant de nombreux exposants et animations festives.\n\nDate : Samedi 16 et dimanche 17 décembre 2023\nLieu: Place du Grand Chêne\nHoraires : 10h00 - 20h00\n\n# Les exposants présents : \n\n- Les Délices de Tante Gertrude - Spécialités alsaciennes et pains d'épices maison\n- La Fabrique du Père Noël - Jouets en bois faits main\n\n\n## - Vins \\& Saveurs d'Hiver - Dégustation de vins chauds et produits du terroir\n\n- Les Lutins du Crochet - Bonnets, écharpes et pulls tricotés avec amour\n- La Forge des Rennes - Sculptures métalliques et décorations artisanales\n- Choco-Miam - Fontaine à chocolat et friandises gourmandes\n- Le Fromage qui Chante - Sélection de fromages affinés avec démonstration musicale $\\boldsymbol{\\mathcal { O }}$\n- La B

## Extraire les textes issus des documents audios

---



In [33]:
# Bouton pour uploader un fichier au début du notebook
from google.colab import files
uploaded = files.upload()

Saving voeux2025_trifouillis.wav to voeux2025_trifouillis (1).wav


In [34]:
# Chemin vers le fichier PDF
audio_path = "/content/voeux2025_trifouillis.wav"

### Whisper

In [35]:
import whisper

# Transcrire avec Whisper
def transcribe_audio_with_whisper(audio_file, model_name="base"):
    # Charger le modèle Whisper
    model = whisper.load_model(model_name)

    # Transcrire le fichier audio
    result = model.transcribe(audio_file)

    # Afficher la transcription
    print("Transcription :")
    print(result["text"])

# Transcrire l'audio avec Whisper
transcribe_audio_with_whisper(audio_path)

100%|███████████████████████████████████████| 139M/139M [00:04<00:00, 33.9MiB/s]


Transcription :
 Chertrifouillesse, chertrifoulesse, est avec une grande émotion, et n'immense hierter, que je me tiens devant vous ce soir, pour vous adresser mes vœux, les plus sincères en ce début d'année. L'année écoulée a été marquée par de nombreux défis. Mais aussi, par de belles avancées, pour notre commune, nous ne pouvons ignorer le contexte régionale et national, qui demeure, complesse et parfois, source d'acquétude, les pays de la loi traversent une période de tension économique et sociale et nous sommes pleinement conscient des préoccupations qui peuvent être les vôtres. Mais je veux vous dire ici, avec force et conviction, nous sommes unif solidaire et nous avons en nous toutes les ressources pour faire face aux épreuves, trifoulis sur l'oi, à toujours suscère, relevé et avancé. Il ne fera pas exception, je sais que la fermeture de notre garde à susciter de vivre, inquétu, et je comprend votre attachement à ce service de transport et sa tiel. Mais le loin de voir cela, co